# Projekt Klasyfikacji Obrazów: Klasyfikator Psów i Kotów

## 1. Opis Projektu
Ten projekt koncentruje się na budowaniu i optymalizacji konwolucyjnej sieci neuronowej (CNN) do klasyfikacji obrazów psów i kotów. Głównym celem jest osiągnięcie wysokiej dokładności przy jednoczesnym eksplorowaniu technik kompresji modelu, takich jak przycinanie (pruning) i kwantyzacja, w celu zmniejszenia rozmiaru modelu i poprawy szybkości wnioskowania.

## 2. Zestaw Danych
Użyty zestaw danych to 'Dogs vs. Cats', składający się z obrazów psów i kotów. Zazwyczaj jest on zorganizowany z oddzielnymi katalogami dla każdej klasy.

## 3. Konfiguracja i Instalacja
Aby uruchomić ten projekt, będziesz potrzebować następujących bibliotek:
- `torch`
- `torchvision`
- `numpy`
- `pandas`
- `matplotlib`
- `seaborn`
- `scikit-learn`
- `zipfile`

```bash
pip install torch torchvision numpy pandas matplotlib seaborn scikit-learn
```

## 4. Przetwarzanie Wstępne Danych
Obrazy są ładowane za pomocą `torchvision.datasets.ImageFolder`. Techniki augmentacji danych są stosowane do zbioru treningowego w celu poprawy uogólniania modelu. Należą do nich:
- Zmiana rozmiaru i losowe kadrowanie
- Losowe odbicia poziome
- Modyfikacja jasności, kontrastu, nasycenia i odcienia (color jittering)
- Losowe obroty
- Rozmycie Gaussa
- Szum Gaussa
- Normalizacja

Zestaw danych jest podzielony na zestawy treningowy, walidacyjny i testowy (odpowiednio 80%, 10%, 10%).

## 5. Architektura Modelu
Projekt wykorzystuje niestandardowy model CNN. Architektura jest następująca:
- **Cechy (Features)**: Sekwencyjny blok warstw konwolucyjnych, normalizacji wsadowej (batch normalization), aktywacji ReLU i warstw max-pooling.
  - 5 bloków, zaczynając od 3 kanałów wejściowych i zwiększając do 512 kanałów wyjściowych.
- **Klasyfikator (Classifier)**: Warstwa adaptacyjnego uśredniania puli (adaptive average pooling), spłaszczanie (flattening), dropout i warstwa liniowa do klasyfikacji binarnej.

Alternatywnie, można użyć wstępnie wytrenowanego modelu ResNet18 jako szkieletu.

## 6. Trening i Ocena
**(Szczegóły pętli treningowej i funkcji straty znalazłyby się tutaj, gdyby były dostępne)**
Modele są oceniane za pomocą różnych metryk:
- Dokładność (Accuracy)
- Precyzja (Precision)
- Czułość (Recall)
- Wynik F1 (F1-Score)
- AUC (Pole pod krzywą charakterystyki operacyjnej odbiornika)
- Czas wnioskowania (ms/partia)
- Rozmiar modelu (MB)
- Rzadkość (Sparsity) (% wag zerowych)

## 7. Techniki Kompresji Modelu

### Przycinanie (Pruning)
Niestrukturyzowane przycinanie L1 jest stosowane globalnie do warstw konwolucyjnych i liniowych w celu wprowadzenia rzadkości, zmniejszając liczbę parametrów i potencjalnie przyspieszając wnioskowanie.

### Kwantyzacja (Quantization)
Dynamiczna kwantyzacja (do `torch.qint8`) jest stosowana do warstw liniowych w celu zmniejszenia zużycia pamięci i potencjalnej poprawy szybkości wnioskowania poprzez użycie typów danych o niższej precyzji.

### Model Przycięty + Skwantyzowany
Zarówno przycinanie, jak i kwantyzacja są łączone w celu osiągnięcia maksymalnej kompresji.

## 8. Wyniki i Porównanie
Poniższa tabela podsumowuje wydajność i metryki rozmiaru różnych wariantów modelu:

| Model            | Rzadkość | Rozmiar (Standardowy MB) | Rozmiar (Skompresowany MB) | Dokładność (%) | Wynik F1 (%) | Średni czas wnioskowania (ms/partia) |
|:-----------------|:---------|:-------------------|:---------------------|:-------------|:-------------|:------------------------------|
| Oryginalny       | 0.0      | 18.03              | 16.66                | 94.28        | 94.42        | 3310.46                       |
| Przycięty        | 49.94    | 18.03              | 9.84                 | 94.24        | 94.30        | 3281.89                       |
| Skwantyzowany    | 0.0      | 18.03              | 16.66                | 94.24        | 94.38        | 3262.53                       |
| Przycięty+Skwantyzowany | 49.94    | 18.03              | 9.84                 | 94.20        | 94.26        | 3296.59                       |

*   **Najlepszy model dla**:
    *   **Najmniejszy rozmiar**: `Przycięty+Skwantyzowany` (9.84 MB)
    *   **Najwyższa dokładność**: `Oryginalny` (94.28%)
    *   **Najszybsza prędkość**: `Skwantyzowany` (3263 ms/partia)
    *   **Najlepsza efektywność (dokładność/rozmiar)**: `Przycięty`

Wyniki te wskazują, że przycinanie i kwantyzacja skutecznie zmniejszają rozmiar modelu przy minimalnym wpływie na dokładność, a także oferują niewielkie poprawy w szybkości wnioskowania dla niektórych konfiguracji. Model `Przycięty+Skwantyzowany` osiągnął najmniejszy rozmiar, co czyni go idealnym do środowisk o ograniczonych zasobach, z niewielkim spadkiem dokładności.

![Results](results.png)